In [1]:
import tensorflow as tf
import numpy as np
import math

SEPCTURAL_SAMPLES = 10
FEATURE_DIM = SEPCTURAL_SAMPLES*6*2  #120
CONV_LEN = 3
CONV_LEN_INTE = 3#4
CONV_LEN_LAST = 3#5
CONV_NUM = 64
CONV_MERGE_LEN = 8
CONV_MERGE_LEN2 = 6
CONV_MERGE_LEN3 = 4
CONV_NUM2 = 64
INTER_DIM = 128
OUT_DIM = 6#len(idDict)
WIDE = 20
CONV_KEEP_PROB = 0.2

BATCH_SIZE = 64
TOTAL_ITER_NUM = 1000

select = 'a'

metaDict = {'a':[119080, 1193], 'b':[116870, 1413], 'c':[116020, 1477]}
TRAIN_SIZE = metaDict[select][0]
EVAL_DATA_SIZE = metaDict[select][1]
EVAL_ITER_NUM = int(math.ceil(EVAL_DATA_SIZE / BATCH_SIZE))


In [2]:
###### Import training data
train_x = np.load('DeepSense_data/train_x.npy')
train_y = np.load('DeepSense_data/train_y.npy')
eval_x = np.load('DeepSense_data/eval_x.npy')
eval_y = np.load('DeepSense_data/eval_y.npy')

# train_x = np.array(train_x)
print(train_x.shape,train_y.shape)
print(eval_x.shape,eval_y.shape)

acc,gyo = np.split(train_x,2)
acc = acc.reshape((-1,20,60,1))
gyo = gyo.reshape((-1,20,60,1))
print(acc.shape,gyo.shape)
print(train_y[0])

(119080, 20, 120) (119080, 6)
(1193, 20, 120) (1193, 6)
(119080, 20, 60, 1) (119080, 20, 60, 1)
[0. 0. 0. 0. 1. 0.]


In [2]:
###### Import training data
train_x = np.load('fre_data.npy')
print(train_x.shape)
train_y = np.load('fre_data_label.npy')
# eval_x = np.load('DeepSense_data/eval_x.npy')
# eval_y = np.load('DeepSense_data/eval_y.npy')
train_x = train_x.reshape((-1,20,120))

# train_x = np.array(train_x)
print(train_x.shape,train_y.shape)
# print(eval_x.shape,eval_y.shape)

acc,gyo = np.split(train_x,2)
acc = acc.reshape((-1,20,60,1))
gyo = gyo.reshape((-1,20,60,1))
print(acc.shape,gyo.shape)
print(train_y[0])

(254000, 120)
(12700, 20, 120) (12700, 6)
(12700, 20, 60, 1) (12700, 20, 60, 1)
[0 0 0 0 1 0]


In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [4]:
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.core import Activation
from keras.layers import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv3D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras import optimizers
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Reshape
import keras.backend as K
from keras.layers.recurrent import GRU,LSTM
from keras.layers import GRUCell,Lambda
from keras.layers import RNN
from keras import regularizers
# from keras.backend import concatenate

Using TensorFlow backend.


In [5]:
acc_inputs, gyro_inputs = acc,gyo
print('acc_inputs.shape,gyro_inputs.shape ',acc_inputs.shape,gyro_inputs.shape)

input1 = Input(shape=(20,120,1), name='input1')
# sensor_inputs shape (BATCH_SIZE, WIDE, FEATURE_DIM, CHANNEL=1)
acc_conv1 = Conv2D(CONV_NUM, kernel_size=[1, 2*3*CONV_LEN],
        strides=(1, 2*3), padding='VALID', activation=None, data_format='channels_last')(input1)
acc_conv1 = BatchNormalization()(acc_conv1)
acc_conv1 = Activation('relu')(acc_conv1)
acc_conv1_shape = acc_conv1.get_shape().as_list()
acc_conv1 = Dropout(CONV_KEEP_PROB, noise_shape=[acc_conv1_shape[0], 1, 1, acc_conv1_shape[3]], seed=None)(acc_conv1)
#         layers.dropout(acc_conv1, CONV_KEEP_PROB, is_training=train,
#             noise_shape=[acc_conv1_shape[0], 1, 1, acc_conv1_shape[3]], scope='acc_dropout1')

acc_conv2 = Conv2D(CONV_NUM, kernel_size=[1, CONV_LEN_INTE],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_last')(acc_conv1)
acc_conv2 = BatchNormalization()(acc_conv2)
acc_conv2 = Activation('relu')(acc_conv2)
acc_conv2_shape = acc_conv2.get_shape().as_list()
acc_conv2 = Dropout(CONV_KEEP_PROB, noise_shape=[acc_conv2_shape[0], 1, 1, acc_conv2_shape[3]], seed=None)(acc_conv2)
#layers.dropout(acc_conv2, CONV_KEEP_PROB, is_training=train,
#             noise_shape=[acc_conv2_shape[0], 1, 1, acc_conv2_shape[3]], scope='acc_dropout2')

acc_conv3 = Conv2D(CONV_NUM, kernel_size=[1, CONV_LEN_LAST],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_last')(acc_conv2)
acc_conv3 = BatchNormalization()(acc_conv3)
acc_conv3 = Activation('relu')(acc_conv3)
acc_conv3_shape = acc_conv3.get_shape().as_list()
print('acc_conv3_shape',acc_conv3_shape)
outputs = Reshape((acc_conv3_shape[1], -1))(acc_conv3)
# tf.reshape(acc_conv3, [acc_conv3_shape[0], acc_conv3_shape[1], 1, acc_conv3_shape[2],acc_conv3_shape[3]])
#print('acc_conv_out.shape ',acc_conv_out.shape)
outputs = GRU(INTER_DIM, activation='tanh',dropout=0.5,return_sequences=True)(outputs)
outputs = Dropout(0.5)(outputs)
outputs = GRU(INTER_DIM, activation='tanh',dropout=0.5,return_sequences=True)(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Flatten()(outputs)
outputs = Dense(6, activation='softmax',kernel_regularizer=regularizers.l2(5e-3))(outputs)

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.5, beta_2=0.9, epsilon=None, decay=0.0, amsgrad=False)

model = Model(inputs=input1, outputs=outputs)
model.compile(loss='categorical_crossentropy',
      optimizer=adam,
      metrics=['accuracy'])

#model.fit([acc_inputs,gyro_inputs], train_y, batch_size=64, epochs=10)
model.summary()

# sensor_conv_in = concatenate([acc_conv_out, gyro_conv_out], axis=2)
# # print('sensor_conv_in ',sensor_conv_in)
# senor_conv_shape = sensor_conv_in.get_shape().as_list()
# sensor_conv_in = Dropout(CONV_KEEP_PROB, noise_shape=[senor_conv_shape[0], 1, 1, 1, senor_conv_shape[4]],seed=None)(sensor_conv_in)
# print('sensor_conv_in ',sensor_conv_in)

# sensor_conv1 = Conv3D(CONV_NUM2, kernel_size=[1, 2, CONV_MERGE_LEN],
#                 strides=(1, 1, 1), padding='SAME', activation=None, data_format='channels_last')(sensor_conv_in)
# sensor_conv1 = BatchNormalization()(sensor_conv1)
# sensor_conv1 = Activation('relu')(sensor_conv1)
# sensor_conv1_shape = sensor_conv1.get_shape().as_list()
# sensor_conv1 = Dropout(CONV_KEEP_PROB, noise_shape=[sensor_conv1_shape[0], 1, 1, 1, sensor_conv1_shape[4]], seed=None)(sensor_conv1)

# sensor_conv2 = Conv3D(CONV_NUM2, kernel_size=[1, 2, CONV_MERGE_LEN2],
#                 strides=(1, 1, 1), padding='SAME', activation=None, data_format='channels_last')(sensor_conv1)
# sensor_conv2 = BatchNormalization()(sensor_conv2)
# sensor_conv2 = Activation('relu')(sensor_conv2)
# sensor_conv2_shape = sensor_conv2.get_shape().as_list()
# sensor_conv2 = Dropout(CONV_KEEP_PROB, 
#                        noise_shape=[sensor_conv2_shape[0], 1, 1, 1, sensor_conv2_shape[4]])(sensor_conv2)

# sensor_conv3 = Conv3D(CONV_NUM2, kernel_size=[1, 2, CONV_MERGE_LEN3],
#                 strides=(1, 1, 1), padding='SAME', activation=None, data_format='channels_last')(sensor_conv2)
# sensor_conv3 = BatchNormalization()(sensor_conv3)
# sensor_conv3 = Activation('relu')(sensor_conv3)
# sensor_conv3_shape = sensor_conv3.get_shape().as_list()
# print('sensor_conv3.shape ',sensor_conv3_shape)
# sensor_conv_out = Reshape(([sensor_conv3_shape[1], sensor_conv3_shape[2]*sensor_conv3_shape[3]*sensor_conv3_shape[4]]))(sensor_conv3)
# #  tf.reshape(sensor_conv3, [sensor_conv3_shape[0], sensor_conv3_shape[1], sensor_conv3_shape[2]*sensor_conv3_shape[3]*sensor_conv3_shape[4]])
# print('sensor_conv_out.shape ',sensor_conv_out.shape)
# # outputs = Flatten()(sensor_conv_out)


W0913 14:55:41.738993  9080 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0913 14:55:41.759994  9080 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0913 14:55:41.761994  9080 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0913 14:55:41.798996  9080 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.comp

acc_inputs.shape,gyro_inputs.shape  (12700, 20, 60, 1) (12700, 20, 60, 1)


W0913 14:55:43.422089  9080 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0913 14:55:43.529095  9080 deprecation.py:506] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


acc_conv3_shape [None, 20, 14, 64]


W0913 14:55:44.711163  9080 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 20, 120, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 18, 64)        1216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 18, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 20, 18, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 18, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 16, 64)        12352     
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 16, 64)        256       
__________

In [8]:
x_train = x_train.reshape((-1,20,120,1))
model.fit(x_train, y_train, batch_size=64,validation_split=0.2, epochs=20,callbacks=[history])

W0913 14:56:23.383163  9080 deprecation.py:323] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 8128 samples, validate on 2032 samples
Epoch 1/20
8128/8128 [==============================] - 29s 4ms/step - loss: 1.5152 - acc: 0.4065 - val_loss: 0.9117 - val_acc: 0.6924
Epoch 2/20
8128/8128 [==============================] - 24s 3ms/step - loss: 0.9191 - acc: 0.6489 - val_loss: 0.5342 - val_acc: 0.8056
Epoch 3/20
8128/8128 [==============================] - 24s 3ms/step - loss: 0.6834 - acc: 0.7480 - val_loss: 0.4632 - val_acc: 0.8337
Epoch 4/20
8128/8128 [==============================] - 25s 3ms/step - loss: 0.5317 - acc: 0.8097 - val_loss: 0.2588 - val_acc: 0.9203
Epoch 5/20
8128/8128 [==============================] - 25s 3ms/step - loss: 0.3963 - acc: 0.8612 - val_loss: 0.1985 - val_acc: 0.9478
Epoch 6/20
8128/8128 [==============================] - 25s 3ms/step - loss: 0.3232 - acc: 0.8951 - val_loss: 0.2842 - val_acc: 0.9154
Epoch 7/20
8128/8128 [==============================] - 25s 3ms/step - loss: 0.2697 - acc: 0.9186 - val_loss: 0.1360 - val_acc: 0.9641
Epoch 8

In [9]:
from keras.models import load_model
 
model.save('DeepSense_CNN.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model
 
# returns a compiled model
# identical to the previous one
#model = load_model('DeepSense_keras.h5')

In [11]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn import metrics

test_x = x_test.reshape((-1,20,120,1))
test_y = y_test
# test_x = np.array(test_x)
# test_y = np.array(test_y)

# print ( "Loss = " + str( preds[0] ) )
# print ( "Test Accuracy = " + str( preds[1] ) )
print(model.metrics_names)
loss, acc = model.evaluate(test_x,test_y, batch_size=128)
print('loss, acc ',loss,acc)


y_pred = model.predict(test_x, batch_size=128, verbose=1)
y_pred = (y_pred > 0.5) 
y_pred_bool = np.argmax(y_pred, axis=1)

for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0

print(classification_report(test_y, y_pred))
print(metrics.f1_score(test_y, y_pred, average="weighted"))
print(confusion_matrix(test_y.argmax(axis=1), y_pred.argmax(axis=1)))

['loss', 'acc']
2540/2540 [==============================] - 1s 480us/step
loss, acc  0.07137528294184077 0.9842519679407435
2540/2540 [==============================] - 2s 596us/step
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       386
           1       0.99      1.00      0.99       500
           2       0.98      0.96      0.97       358
           3       0.98      0.97      0.98       399
           4       0.99      1.00      0.99       417
           5       0.97      0.98      0.97       480

   micro avg       0.98      0.99      0.98      2540
   macro avg       0.98      0.98      0.98      2540
weighted avg       0.98      0.99      0.98      2540
 samples avg       0.98      0.99      0.98      2540

0.9815384530524908
[[384   1   0   0   1   0]
 [  0 499   0   0   1   0]
 [  2   0 342   3   0  11]
 [  9   0   2 386   0   2]
 [  0   1   0   0 416   0]
 [  1   0   2   5   0 472]]


In [7]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
        self.accuracy = {'batch': [], 'epoch': []}
        self.val_loss = {'batch': [], 'epoch': []}
        self.val_acc = {'batch': [], 'epoch': []}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        #创建一个图
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')#plt.plot(x,y)，这个将数据画成曲线
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)#设置网格形式
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')#给x，y轴加注释
        plt.legend(loc="upper right")#设置图例显示位置
        plt.show()

history = LossHistory()


In [10]:
history.loss_plot('epoch')

AttributeError: 'LossHistory' object has no attribute 'losses'